# Massachusetts Scorecard

This notebook generates the county, city, and statewide "future voter scorecards" for MA. It is generalize to be updated every month, with minimal changes.

Scorecard outputs (tables) are written back to BigQuery, where they are then read into Google Sheets for formatting

# Data and methodology notes


- MA provided counts of the preregistrants that signed up each month. To estimate the number of preregistered 16 and 17 year olds as of a given month, we look at the sign up counts for the past 24 months, and apply a discount to each month, assuming an even distribution of possible 18th birthdays across the year. Because we don't know when in the month each preregistrant signed up, we assume everyone signs up in the middle of the month (tactically, we calculate a discount rate assuming everyone signed up on the first of the month, and then a second rate assuming everyone signed up at the end of the month, and we average those rates.)

All to say! These are rough estimates!

- MA did not provide prereg counts for unincorporated areas of counties (the county areas not in cities/towns). Therefor, the State and County numbers (both numerator and denominator) are aggregated from town and city data

In [21]:
import pandas as pd
import numpy as np
import pandas_gbq

## Inputs
Update the fields below each month

In [22]:
# Inputs
as_of_data = "2024-05-01"
as_of_data_date = pd.Timestamp(as_of_data)
acs_year = '2022' # 2022 for 2023 scorecards. ACS vintages trail by 2 years
table_suffix = '201608_202405'


In [23]:
as_of_data_date = pd.Timestamp(as_of_data)
data_date_suffix = str(as_of_data_date.year)+ str(as_of_data_date.month).rjust(2, "0")


## Outputs
Run the cells below, without edits each month

### County Scorecard

In [24]:
# Define table names
voter_file_table = "monthly_prereg_counts_by_city_and_county_" + table_suffix
acs_S0101_table = "S0101_ma_county_subdivision_acs5y_" + acs_year

In [25]:
voter_file_table

'monthly_prereg_counts_by_city_and_county_201608_202405'

#### Query from BQ
This query:
* Takes the summary of 18yo registrants we get from the SOS
* Then, left joins the county estimates for the total number of 17, 18, and 19yos from the ACS
    * The estimates for the total number of 17-19yo yos are derived from the raw estimates of 15-17 yos, **assuming a uniform distribution of population across 15, 16, and 17 year olds.**
    * Since the ACS trails by 2 years, the ACS estimate of 15-17yos is used as a proxy for the number of 17-19yos today. (This means we are intentionally *not* trying to count the college student or "group quarters" population in our denominator)

* Ages are calculated as of the date of the voter file and as of the next general election 

In [26]:
# Define GCP project
project_id = "tcc-research"

# Define query, including variables and column names that adjust with time
sql= """

WITH month_number AS(
  SELECT 
  *,
  CASE 
    WHEN MONTH = "January" THEN 1
    WHEN MONTH = "February" THEN 2
    WHEN MONTH = "March" THEN 3
    WHEN MONTH = "April" THEN 4
    WHEN MONTH = "May" THEN 5
    WHEN MONTH = "June" THEN 6
    WHEN MONTH = "July" THEN 7
    WHEN MONTH = "August" THEN 8
    WHEN MONTH = "September" THEN 9
    WHEN MONTH = "October" THEN 10
    WHEN MONTH = "November" THEN 11
    WHEN MONTH = "December" THEN 12
  END AS MONTH_NUM,
 FROM `tcc-research.ma_sources.""" + voter_file_table + """`
 
), month_offset AS(

  SELECT
  *,
  RANK() OVER (PARTITION BY CITY_OR_TOWN, COUNTY ORDER BY YEAR DESC, MONTH_NUM DESC ) AS NTH_MONTH
  -- MOD(24 - (5- MONTH_NUM),12) AS MONTH_OFFSET,
  -- 12 - MOD(12 - (5- MONTH_NUM),12) AS DIFF_TO_MONTH_OFFSET,
  -- NTH_MONTH/ 24 AS PREREG_DISCOUNT_RATE,
  FROM month_number
  WHERE (DATE(YEAR, MONTH_NUM, 1)) <= '""" + as_of_data + """'

), prereg_est AS(

SELECT
*,
NTH_MONTH/ 24 AS PREREG_DISCOUNT_RATE_HIGH,
(NTH_MONTH-1)/ 24 AS PREREG_DISCOUNT_RATE_LOW,
PREREG_COUNT * (1- (NTH_MONTH/ 24)) AS MONTHLY_DISCOUNTED_PREREG_HIGH,
PREREG_COUNT * (1- ((NTH_MONTH-1)/ 24)) AS MONTHLY_DISCOUNTED_PREREG_LOW,
SUM(PREREG_COUNT * (1- (NTH_MONTH/ 24))) OVER (PARTITION BY CITY_OR_TOWN, COUNTY ORDER BY YEAR, MONTH_NUM ) AS EST_PREREG_IN_YEAR_HIGH,
SUM(PREREG_COUNT * (1- ((NTH_MONTH-1)/ 24))) OVER (PARTITION BY CITY_OR_TOWN, COUNTY ORDER BY YEAR, MONTH_NUM ) AS EST_PREREG_IN_YEAR_LOW
FROM month_offset 
WHERE NTH_MONTH <= 24
ORDER BY CITY_OR_TOWN, COUNTY, YEAR DESC, MONTH_NUM DESC

), prereg_nums AS(

SELECT
*
FROM prereg_est
WHERE (DATE(YEAR, MONTH_NUM, 1)) = '""" + as_of_data + """'

), ma_towns AS(
  SELECT
    STATE_FIPS,
    REGEXP_EXTRACT(GEO_ID, r"(\d{5})$") AS PLACE_FIPS,
    TRIM(REGEXP_REPLACE(UPPER(REGEXP_EXTRACT(NAME, 	r"^(.*)(?: city,| town,)")), 	r"( TOWN)$", "")) AS CITY_OR_TOWN,
    EST_15_TO_17_YO,
  FROM`acs_sources.""" + acs_S0101_table + """` 

), prereg_combo AS(

SELECT 
a.CITY_OR_TOWN,
a.COUNTY,
a.YEAR,
a.MONTH,
b.STATE_FIPS,
b.PLACE_FIPS,
a.EST_PREREG_IN_YEAR_HIGH,
a.EST_PREREG_IN_YEAR_LOW, 
b.EST_15_TO_17_YO,
b.EST_15_TO_17_YO / 3 *2 AS EST_16_AND_17_YO
 FROM prereg_nums a LEFT JOIN ma_towns b ON TRIM(a.CITY_OR_TOWN) = b.CITY_OR_TOWN

)

SELECT
COUNTY,
YEAR,
MONTH,
STATE_FIPS,
SUM(EST_PREREG_IN_YEAR_HIGH) AS EST_PREREG_IN_YEAR_HIGH,
SUM(EST_PREREG_IN_YEAR_LOW) AS EST_PREREG_IN_YEAR_LOW, 
SUM(EST_15_TO_17_YO) AS EST_15_TO_17_YO,
SUM(EST_15_TO_17_YO / 3 * 2) AS EST_16_AND_17_YO
FROM prereg_combo
GROUP BY COUNTY, YEAR, MONTH, STATE_FIPS
"""

df = pandas_gbq.read_gbq(sql, project_id=project_id)

<>:56: SyntaxWarning: invalid escape sequence '\d'
<>:56: SyntaxWarning: invalid escape sequence '\d'
/var/folders/31/6hdc7tpn2bvgc_bm41yw6qp80000gp/T/ipykernel_56362/864229508.py:56: SyntaxWarning: invalid escape sequence '\d'
  WHERE (DATE(YEAR, MONTH_NUM, 1)) = '""" + as_of_data + """'
/Users/sarah/Documents/GitHub/TCC/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:2257: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [27]:
# Preview
df.head()

,COUNTY,YEAR,MONTH,STATE_FIPS,EST_PREREG_IN_YEAR_HIGH,EST_PREREG_IN_YEAR_LOW,EST_15_TO_17_YO,EST_16_AND_17_YO
0,NORFOLK,2024,May,25,7287.750000,7898.041667,28061,18707.333333
1,BARNSTABLE,2024,May,25,1720.625000,1862.958333,6631,4420.666667
2,BERKSHIRE,2024,May,25,1039.791667,1126.916667,4276,2850.666667
3,SUFFOLK,2024,May,25,3467.833333,3755.125000,21046,14030.666667
4,ESSEX,2024,May,25,7714.000000,8346.583333,31032,20688.000000


In [28]:
df_reg_est = df.copy()

#### Metric 1: Estimated registration rate of 18 and 19 year olds as of a rolling date (i.e. latest month)


In [29]:
df_reg_est['N_EST_VOTERS_16_17'] = (df_reg_est.EST_PREREG_IN_YEAR_LOW + df_reg_est.EST_PREREG_IN_YEAR_HIGH) / 2
df_reg_est['EST_REG_RATE_16_17_YO'] = df_reg_est['N_EST_VOTERS_16_17'] / df_reg_est['EST_16_AND_17_YO']
df_reg_est.sort_values('EST_15_TO_17_YO', ascending=False)

,COUNTY,YEAR,MONTH,STATE_FIPS,EST_PREREG_IN_YEAR_HIGH,EST_PREREG_IN_YEAR_LOW,EST_15_TO_17_YO,EST_16_AND_17_YO,N_EST_VOTERS_16_17,EST_REG_RATE_16_17_YO
10,MIDDLESEX,2024,May,25,14343.666667,15548.791667,56522,37681.333333,14946.229167,0.396648
6,WORCESTER,2024,May,25,8585.041667,9293.333333,33291,22194.000000,8939.187500,0.402775
4,ESSEX,2024,May,25,7714.000000,8346.583333,31032,20688.000000,8030.291667,0.388162
0,NORFOLK,2024,May,25,7287.750000,7898.041667,28061,18707.333333,7592.895833,0.405878
9,BRISTOL,2024,May,25,5612.916667,6073.958333,21957,14638.000000,5843.437500,0.399196
12,PLYMOUTH,2024,May,25,5631.583333,6103.125000,21493,14328.666667,5867.354167,0.409484
3,SUFFOLK,2024,May,25,3467.833333,3755.125000,21046,14030.666667,3611.479167,0.257399
13,HAMPDEN,2024,May,25,4485.250000,4853.583333,18330,12220.000000,4669.416667,0.382113
1,BARNSTABLE,2024,May,25,1720.625000,1862.958333,6631,4420.666667,1791.791667,0.405322
7,HAMPSHIRE,2024,May,25,1222.666667,1326.958333,4767,3178.000000,1274.812500,0.401137


#### Output
Write back to BQ

In [30]:
# write
project_id = "tcc-research"
table_id = 'ma_output.' + data_date_suffix+ '_ma_county_scorecard_output'

pandas_gbq.to_gbq(df_reg_est, table_id, project_id=project_id, if_exists='replace')

### City/Town Scorecard

In [31]:
# Define table names
voter_file_table = "monthly_prereg_counts_by_city_and_county_" + table_suffix
acs_S0101_table = "S0101_ma_county_subdivision_acs5y_" + acs_year

#### Query from BQ
This query:
* Takes the summary of 18yo registrants we get from the SOS
* Then, left joins the county estimates for the total number of 17, 18, and 19yos from the ACS
    * The estimates for the total number of 17-19yo yos are derived from the raw estimates of 15-17 yos, **assuming a uniform distribution of population across 15, 16, and 17 year olds.**
    * Since the ACS trails by 2 years, the ACS estimate of 15-17yos is used as a proxy for the number of 17-19yos today. (This means we are intentionally *not* trying to count the college student or "group quarters" population in our denominator)

* Ages are calculated as of the date of the voter file and as of the next general election 

In [32]:
# Define GCP project
project_id = "tcc-research"

# Define query, including variables and column names that adjust with time
sql= """

WITH month_number AS(
  SELECT 
  *,
  CASE 
    WHEN MONTH = "January" THEN 1
    WHEN MONTH = "February" THEN 2
    WHEN MONTH = "March" THEN 3
    WHEN MONTH = "April" THEN 4
    WHEN MONTH = "May" THEN 5
    WHEN MONTH = "June" THEN 6
    WHEN MONTH = "July" THEN 7
    WHEN MONTH = "August" THEN 8
    WHEN MONTH = "September" THEN 9
    WHEN MONTH = "October" THEN 10
    WHEN MONTH = "November" THEN 11
    WHEN MONTH = "December" THEN 12
  END AS MONTH_NUM,
 FROM `tcc-research.ma_sources.""" + voter_file_table + """`
 
), month_offset AS(

  SELECT
  *,
  RANK() OVER (PARTITION BY CITY_OR_TOWN, COUNTY ORDER BY YEAR DESC, MONTH_NUM DESC ) AS NTH_MONTH
  -- MOD(24 - (5- MONTH_NUM),12) AS MONTH_OFFSET,
  -- 12 - MOD(12 - (5- MONTH_NUM),12) AS DIFF_TO_MONTH_OFFSET,
  -- NTH_MONTH/ 24 AS PREREG_DISCOUNT_RATE,
  FROM month_number
  WHERE (DATE(YEAR, MONTH_NUM, 1)) <= '""" + as_of_data + """'

), prereg_est AS(

SELECT
*,
NTH_MONTH/ 24 AS PREREG_DISCOUNT_RATE_HIGH,
(NTH_MONTH-1)/ 24 AS PREREG_DISCOUNT_RATE_LOW,
PREREG_COUNT * (1- (NTH_MONTH/ 24)) AS MONTHLY_DISCOUNTED_PREREG_HIGH,
PREREG_COUNT * (1- ((NTH_MONTH-1)/ 24)) AS MONTHLY_DISCOUNTED_PREREG_LOW,
SUM(PREREG_COUNT * (1- (NTH_MONTH/ 24))) OVER (PARTITION BY CITY_OR_TOWN, COUNTY ORDER BY YEAR, MONTH_NUM ) AS EST_PREREG_IN_YEAR_HIGH,
SUM(PREREG_COUNT * (1- ((NTH_MONTH-1)/ 24))) OVER (PARTITION BY CITY_OR_TOWN, COUNTY ORDER BY YEAR, MONTH_NUM ) AS EST_PREREG_IN_YEAR_LOW
FROM month_offset 
WHERE NTH_MONTH <= 24
ORDER BY CITY_OR_TOWN, COUNTY, YEAR DESC, MONTH_NUM DESC

), prereg_nums AS(

SELECT
*
FROM prereg_est
WHERE (DATE(YEAR, MONTH_NUM, 1)) = '""" + as_of_data + """'

), ma_towns AS(
  SELECT
    STATE_FIPS,
    REGEXP_EXTRACT(GEO_ID, r"(\d{5})$") AS PLACE_FIPS,
    TRIM(REGEXP_REPLACE(UPPER(REGEXP_EXTRACT(NAME, 	r"^(.*)(?: city,| town,)")), 	r"( TOWN)$", "")) AS CITY_OR_TOWN,
    EST_15_TO_17_YO,
  FROM`acs_sources.""" + acs_S0101_table + """` 

), prereg_combo AS(

SELECT 
a.CITY_OR_TOWN,
a.COUNTY,
a.YEAR,
a.MONTH,
b.STATE_FIPS,
b.PLACE_FIPS,
a.EST_PREREG_IN_YEAR_HIGH,
a.EST_PREREG_IN_YEAR_LOW, 
b.EST_15_TO_17_YO,
b.EST_15_TO_17_YO / 3 *2 AS EST_16_AND_17_YO
 FROM prereg_nums a LEFT JOIN ma_towns b ON TRIM(a.CITY_OR_TOWN) = b.CITY_OR_TOWN

)

SELECT
a.*,
IF(b.CITIES_AND_TOWNS IS NULL,0,1) AS is_greater_boston
FROM prereg_combo a LEFT JOIN `tcc-research.ma_sources.list_of_cities_and_towns_in_greater_boston` b ON a.CITY_OR_TOWN = b.CITIES_AND_TOWNS


"""

df = pandas_gbq.read_gbq(sql, project_id=project_id)

<>:56: SyntaxWarning: invalid escape sequence '\d'
<>:56: SyntaxWarning: invalid escape sequence '\d'
/var/folders/31/6hdc7tpn2bvgc_bm41yw6qp80000gp/T/ipykernel_56362/2059425293.py:56: SyntaxWarning: invalid escape sequence '\d'
  WHERE (DATE(YEAR, MONTH_NUM, 1)) = '""" + as_of_data + """'
/Users/sarah/Documents/GitHub/TCC/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:2257: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [33]:
# Preview
df.head()

,CITY_OR_TOWN,COUNTY,YEAR,MONTH,STATE_FIPS,PLACE_FIPS,EST_PREREG_IN_YEAR_HIGH,EST_PREREG_IN_YEAR_LOW,EST_15_TO_17_YO,EST_16_AND_17_YO,is_greater_boston
0,CANTON,NORFOLK,2024,May,25,11315,273.083333,295.125000,923,615.333333,1
1,CHARLEMONT,FRANKLIN,2024,May,25,12505,6.333333,6.916667,24,16.000000,0
2,REHOBOTH,BRISTOL,2024,May,25,56375,158.583333,171.250000,480,320.000000,0
3,SOUTHWICK,HAMPDEN,2024,May,25,65825,80.583333,87.833333,348,232.000000,0
4,STOUGHTON,NORFOLK,2024,May,25,67945,288.125000,310.666667,1089,726.000000,0


In [34]:
df_reg_est = df.copy()

#### Metric 1: Estimated registration rate of 18 and 19 year olds as of a rolling date (i.e. latest month)


In [35]:
df_reg_est['N_EST_VOTERS_16_17'] = (df_reg_est.EST_PREREG_IN_YEAR_LOW + df_reg_est.EST_PREREG_IN_YEAR_HIGH) / 2
df_reg_est['EST_REG_RATE_16_17_YO'] = df_reg_est['N_EST_VOTERS_16_17'] / df_reg_est['EST_16_AND_17_YO']
df_reg_est.sort_values('EST_15_TO_17_YO', ascending=False)

,CITY_OR_TOWN,COUNTY,YEAR,MONTH,STATE_FIPS,PLACE_FIPS,EST_PREREG_IN_YEAR_HIGH,EST_PREREG_IN_YEAR_LOW,EST_15_TO_17_YO,EST_16_AND_17_YO,is_greater_boston,N_EST_VOTERS_16_17,EST_REG_RATE_16_17_YO
311,BOSTON,SUFFOLK,2024,May,25,07000,2654.666667,2876.416667,16519,11012.666667,1,2765.541667,0.251124
307,WORCESTER,WORCESTER,2024,May,25,82000,1603.291667,1729.041667,7135,4756.666667,0,1666.166667,0.350280
175,SPRINGFIELD,HAMPDEN,2024,May,25,67000,1530.166667,1654.250000,5986,3990.666667,0,1592.208333,0.398983
187,BROCKTON,PLYMOUTH,2024,May,25,09000,890.208333,960.166667,5051,3367.333333,0,925.187500,0.274754
317,LYNN,ESSEX,2024,May,25,37490,818.833333,882.791667,4592,3061.333333,1,850.812500,0.277922
...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,HAWLEY,FRANKLIN,2024,May,25,29475,0.750000,0.916667,3,2.000000,0,0.833333,0.416667
229,MOUNT WASHINGTON,BERKSHIRE,2024,May,25,43300,0.750000,0.833333,0,0.000000,0,0.791667,inf
257,TRURO,BARNSTABLE,2024,May,25,70605,14.666667,15.833333,0,0.000000,0,15.250000,inf
267,GOSNOLD,DUKES,2024,May,25,26325,0.291667,0.333333,0,0.000000,0,0.312500,inf


In [36]:
# Flag low income towns
# 10 lowest income towns

low_income = ['LYNN', 'ORANGE', 'ATHOL', 'FALL RIVER', 'CHELSEA', 'NEW BEDFORD', 'AMHERST', 'HOLYOKE', 'LAWRENCE', 'SPRINGFIELD']
df_reg_est['is_in_10_lowest_income'] = np.where(df_reg_est.CITY_OR_TOWN.isin(low_income),1,0)


In [37]:
# Flag largest cities
df_reg_est['is_in_10_largest'] = np.where(df_reg_est.CITY_OR_TOWN.isin(df_reg_est.nlargest(10, columns='EST_15_TO_17_YO').CITY_OR_TOWN),1,0)
df_reg_est['is_in_20_largest'] = np.where(df_reg_est.CITY_OR_TOWN.isin(df_reg_est.nlargest(20, columns='EST_15_TO_17_YO').CITY_OR_TOWN),1,0)

#### Output
Write back to BQ

In [38]:
# write
project_id = "tcc-research"
table_id = 'ma_output.' + data_date_suffix+ '_ma_city_scorecard_output'

pandas_gbq.to_gbq(df_reg_est, table_id, project_id=project_id, if_exists='replace')

### Statewide Scorecard

In [39]:
# Define table names
voter_file_table = "monthly_prereg_counts_by_city_and_county_" + table_suffix
acs_S0101_table = "S0101_ma_county_subdivision_acs5y_" + acs_year

#### Query from BQ
This query:
* Takes the summary of 18yo registrants we get from the SOS
* Then, left joins the county estimates for the total number of 17, 18, and 19yos from the ACS
    * The estimates for the total number of 17-19yo yos are derived from the raw estimates of 15-17 yos, **assuming a uniform distribution of population across 15, 16, and 17 year olds.**
    * Since the ACS trails by 2 years, the ACS estimate of 15-17yos is used as a proxy for the number of 17-19yos today. (This means we are intentionally *not* trying to count the college student or "group quarters" population in our denominator)

* Ages are calculated as of the date of the voter file and as of the next general election 

In [40]:
# Define GCP project
project_id = "tcc-research"

# Define query, including variables and column names that adjust with time
sql= """

WITH month_number AS(
  SELECT 
  *,
  CASE 
    WHEN MONTH = "January" THEN 1
    WHEN MONTH = "February" THEN 2
    WHEN MONTH = "March" THEN 3
    WHEN MONTH = "April" THEN 4
    WHEN MONTH = "May" THEN 5
    WHEN MONTH = "June" THEN 6
    WHEN MONTH = "July" THEN 7
    WHEN MONTH = "August" THEN 8
    WHEN MONTH = "September" THEN 9
    WHEN MONTH = "October" THEN 10
    WHEN MONTH = "November" THEN 11
    WHEN MONTH = "December" THEN 12
  END AS MONTH_NUM,
 FROM `tcc-research.ma_sources.""" + voter_file_table + """`
 
), month_offset AS(

  SELECT
  *,
  RANK() OVER (PARTITION BY CITY_OR_TOWN, COUNTY ORDER BY YEAR DESC, MONTH_NUM DESC ) AS NTH_MONTH
  -- MOD(24 - (5- MONTH_NUM),12) AS MONTH_OFFSET,
  -- 12 - MOD(12 - (5- MONTH_NUM),12) AS DIFF_TO_MONTH_OFFSET,
  -- NTH_MONTH/ 24 AS PREREG_DISCOUNT_RATE,
  FROM month_number
  WHERE (DATE(YEAR, MONTH_NUM, 1)) <= '""" + as_of_data + """'

), prereg_est AS(

SELECT
*,
NTH_MONTH/ 24 AS PREREG_DISCOUNT_RATE_HIGH,
(NTH_MONTH-1)/ 24 AS PREREG_DISCOUNT_RATE_LOW,
PREREG_COUNT * (1- (NTH_MONTH/ 24)) AS MONTHLY_DISCOUNTED_PREREG_HIGH,
PREREG_COUNT * (1- ((NTH_MONTH-1)/ 24)) AS MONTHLY_DISCOUNTED_PREREG_LOW,
SUM(PREREG_COUNT * (1- (NTH_MONTH/ 24))) OVER (PARTITION BY CITY_OR_TOWN, COUNTY ORDER BY YEAR, MONTH_NUM ) AS EST_PREREG_IN_YEAR_HIGH,
SUM(PREREG_COUNT * (1- ((NTH_MONTH-1)/ 24))) OVER (PARTITION BY CITY_OR_TOWN, COUNTY ORDER BY YEAR, MONTH_NUM ) AS EST_PREREG_IN_YEAR_LOW
FROM month_offset 
WHERE NTH_MONTH <= 24
ORDER BY CITY_OR_TOWN, COUNTY, YEAR DESC, MONTH_NUM DESC

), prereg_nums AS(

SELECT
*
FROM prereg_est
WHERE (DATE(YEAR, MONTH_NUM, 1)) = '""" + as_of_data + """'

), ma_towns AS(
  SELECT
    STATE_FIPS,
    REGEXP_EXTRACT(GEO_ID, r"(\d{5})$") AS PLACE_FIPS,
    TRIM(REGEXP_REPLACE(UPPER(REGEXP_EXTRACT(NAME, 	r"^(.*)(?: city,| town,)")), 	r"( TOWN)$", "")) AS CITY_OR_TOWN,
    EST_15_TO_17_YO,
  FROM`acs_sources.""" + acs_S0101_table + """` 

), prereg_combo AS(

SELECT 
a.CITY_OR_TOWN,
a.COUNTY,
a.YEAR,
a.MONTH,
b.STATE_FIPS,
b.PLACE_FIPS,
a.EST_PREREG_IN_YEAR_HIGH,
a.EST_PREREG_IN_YEAR_LOW, 
b.EST_15_TO_17_YO,
b.EST_15_TO_17_YO / 3 *2 AS EST_16_AND_17_YO
 FROM prereg_nums a LEFT JOIN ma_towns b ON TRIM(a.CITY_OR_TOWN) = b.CITY_OR_TOWN

)

SELECT
STATE_FIPS,
YEAR,
MONTH,
SUM(EST_PREREG_IN_YEAR_HIGH) AS EST_PREREG_IN_YEAR_HIGH,
SUM(EST_PREREG_IN_YEAR_LOW) AS EST_PREREG_IN_YEAR_LOW, 
SUM(EST_15_TO_17_YO) AS EST_15_TO_17_YO,
SUM(EST_15_TO_17_YO / 3 * 2) AS EST_16_AND_17_YO
FROM prereg_combo
GROUP BY STATE_FIPS, YEAR, MONTH
"""

df = pandas_gbq.read_gbq(sql, project_id=project_id)

<>:56: SyntaxWarning: invalid escape sequence '\d'
<>:56: SyntaxWarning: invalid escape sequence '\d'
/var/folders/31/6hdc7tpn2bvgc_bm41yw6qp80000gp/T/ipykernel_56362/1830830620.py:56: SyntaxWarning: invalid escape sequence '\d'
  WHERE (DATE(YEAR, MONTH_NUM, 1)) = '""" + as_of_data + """'
/Users/sarah/Documents/GitHub/TCC/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:2257: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [41]:
# Preview
df.head()

,STATE_FIPS,YEAR,MONTH,EST_PREREG_IN_YEAR_HIGH,EST_PREREG_IN_YEAR_LOW,EST_15_TO_17_YO,EST_16_AND_17_YO
0,25,2024,May,61997.875,67148.291667,251229,167486.0


In [42]:
df_reg_est = df.copy()

#### Metric 1: Estimated registration rate of 18 and 19 year olds as of a rolling date (i.e. latest month)


In [43]:
df_reg_est['N_EST_VOTERS_16_17'] = (df_reg_est.EST_PREREG_IN_YEAR_LOW + df_reg_est.EST_PREREG_IN_YEAR_HIGH) / 2
df_reg_est['EST_REG_RATE_16_17_YO'] = df_reg_est['N_EST_VOTERS_16_17'] / df_reg_est['EST_16_AND_17_YO']
df_reg_est.sort_values('EST_15_TO_17_YO', ascending=False)

,STATE_FIPS,YEAR,MONTH,EST_PREREG_IN_YEAR_HIGH,EST_PREREG_IN_YEAR_LOW,EST_15_TO_17_YO,EST_16_AND_17_YO,N_EST_VOTERS_16_17,EST_REG_RATE_16_17_YO
0,25,2024,May,61997.875,67148.291667,251229,167486.0,64573.083333,0.385543


#### Output
Write back to BQ

In [44]:
# write
project_id = "tcc-research"
table_id = 'ma_output.' + data_date_suffix+ '_ma_statewide_scorecard_output'

pandas_gbq.to_gbq(df_reg_est, table_id, project_id=project_id, if_exists='replace')